<a href="https://colab.research.google.com/github/aislinblack/CS6120-NLP-Project/blob/main/albert/Albert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/huggingface/transformers \
&& cd transformers \
&& git checkout a3085020ed0d81d4903c50967687192e3101e770 

!pip3 install ./transformers

Cloning into 'transformers'...
remote: Enumerating objects: 105690, done.
remote: Counting objects: 100% (357/357), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 105690 (delta 199), reused 281 (delta 159), pack-reused 105333
Receiving objects: 100% (105690/105690), 98.07 MiB | 24.47 MiB/s, done.
Resolving deltas: 100% (78045/78045), done.
Note: checking out 'a3085020ed0d81d4903c50967687192e3101e770'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at a3085020e Added repetition penalty to PPLM example (#2436)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/si

In [2]:
!pip3 install evaluate
!pip3 install bert_score
!pip3 install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 255 kB/s 
     |████████████████████████████████| 212 kB 32.3 MB/s 
     |████████████████████████████████| 115 kB 64.5 MB/s 
     |████████████████████████████████| 101 kB 10.4 MB/s 
     |████████████████████████████████| 141 kB 41.5 MB/s 
     |████████████████████████████████| 365 kB 54.8 MB/s 
     |████████████████████████████████| 596 kB 56.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 4.8 MB/s 
     |████████████████████████████████| 4.7 MB 26.7 MB/s 
     |████████████████████████████████| 6.6 MB 61.1 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizer

In [13]:
from transformers import pipeline

from transformers import (
    AlbertConfig,
    AlbertForQuestionAnswering,
    AlbertTokenizer,
    AlbertTokenizerFast,
    squad_convert_examples_to_features
)

import json
import numpy as np
import time

In [16]:
## use this flag to indicate if you want to run the predictions yourself
## otherwise they will be loaded from files
bRunPred = False

modelname = 'twmkn9/albert-base-v2-squad2'
model = AlbertForQuestionAnswering.from_pretrained(modelname)
tokenizer = AlbertTokenizerFast.from_pretrained(modelname)  


In [17]:
test_file = open('Covid-QA-more-focused.json')
test_data = json.load(test_file)
print(len(test_data['data']))

42


In [18]:
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer, device=0, batch_size=32)

In [19]:
bRunPred = True

if bRunPred==True:
    num_questions = 0
    num_correct_pred = 0
    questions = []
    answers = []
    predictions = []
    total_time = time.time()

    for i in test_data['data']:
      para = i['paragraphs'][0]
      t_paragraph = time.time()
      #print('-----------------')
      #print(para.keys())
      print()
      qas = para['qas']
      context = para['context']
      for eachQA in qas:
        num_questions+=1
        print('num_ques ', num_questions)
        #print('eachQA ', eachQA)
        print('Question is: ', eachQA['question'])
        print('Given Answer (GT) is: ', eachQA['answers'][0]['text'])
        questions.append(eachQA['question'])
        answers.append(eachQA['answers'][0]['text'])

        answer = nlp({
            'question': eachQA['question'],
            'context': context})
        print('Predicted answer is: ', answer['answer'])
        predictions.append(answer['answer'])
        #print('answer ', answer)

        if answer['score'] > 0.1:
          if len(answer['answer']) <= len(eachQA['answers'][0]['text']) + 1:
            if len(answer['answer']) == len(eachQA['answers'][0]['text']) + 1:
              answer['answer'] = answer['answer'][:-1]

          if answer['answer'] in eachQA['answers'][0]['text']:
            num_correct_pred+=1
            print('Correct !!!')
      print('Time for para:', time.time() - t_paragraph)

    print('Time taken: ', time.time() - total_time)



    print('\n\n Accuracy is ', num_correct_pred/num_questions) # This accuracy is manually calculated


num_ques  1
Question is:  How many surgical masks or respirators have past studies projected will be required for a pandemic in the United States?
Given Answer (GT) is:  an estimated 7.3 billion
Predicted answer is:   7.3 billion
Correct !!!
num_ques  2
Question is:  What is the acronym MERS-CoV?
Given Answer (GT) is:  Middle East respiratory syndrome coronavirus
Predicted answer is:   Middle East respiratory syndrome
num_ques  3
Question is:  What are the critical factors that determine the effect of an epidemic?
Given Answer (GT) is:  Transmissibility and severity
Predicted answer is:   Model parameters and outbreak control
num_ques  4
Question is:  When did the World Health Organization (WHO) officially declare the 2019-nCoV epidemic as a Public Health Emergency of International Concern?
Given Answer (GT) is:  January 30, 2020
Predicted answer is:   January 30, 2020,
num_ques  5
Question is:  What influenza virus was identified in China in 2013?
Given Answer (GT) is:  H7N9
Predicte

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


Predicted answer is:  
HMPV
num_ques  12
Question is:  Are there any vaccines against to protect against respiratory viral infections?
Given Answer (GT) is:  Currently, there are no approved vaccines or medications available for most of the respiratory viruses
Predicted answer is:   no approved vaccines or medications available
Correct !!!
Time for para: 1.1048636436462402

num_ques  13
Question is:  What types of acute respiratory infections can be screened and diagnosed with multiplex PCR?
Given Answer (GT) is:  influenza and parainfluenza viruses, RSV, adenovirus, rhinovirus, human metapneumovirus, enterovirus, bocavirus and coronavirus
Predicted answer is:   influenza virus and RSV infections
Time for para: 0.3524508476257324

num_ques  14
Question is:  What is the role of antibodies during infection?
Given Answer (GT) is:  Antibodies against foreign antigens are a critical component of the overall immune response and can facilitate pathogen clearance during a primary infection and

In [20]:
if bRunPred==True:
    print(len(questions)) 
    print(len(answers))
    print(len(predictions))
    print(num_correct_pred)

828
828
828
157


In [21]:
if bRunPred==True:
    with open("answers_albert.json", "w") as f:
        json.dump(answers, f)

    with open("questions_albert.json", "w") as g:
        json.dump(questions, g)

    with open("predictions_albert.json", "w") as h:
        json.dump(predictions, h)    

In [ ]:
if bRunPred==False:
    answers = open('answers_albert.json')
    answers = json.load(answers)
    print(len(answers))

    predictions = open('predictions_albert.json')
    predictions = json.load(predictions)
    print(len(predictions))

In [22]:
from evaluate import load
import pandas as pd


In [23]:
bertscore = load("bertscore") #('exact_match')

results = bertscore.compute(predictions=predictions, references=answers, lang="en")
print(results.keys())

dict_keys(['precision', 'recall', 'f1', 'hashcode'])


In [24]:
print(len(results['f1']))
import numpy as np
print(np.mean(results['precision']))
print(np.mean(results['f1']))
print(np.mean(results['recall']))

828
0.9085662393034368
0.8895730764969535
0.8724355256355903


Using lang='en',

Precision: 0.910

F1: 0.890

Recall: 0.872
